# Chain Of Thought (COT)

**What is Chain Of Thought?**

Consider one’s own thought process when solving a complicated reasoning task such as a multi-step math word problem. It is typical to decompose the problem into intermediate steps and solve each before giving the final answer: “After Jane gives 2 flowers to her mom she has 10 . . . then after she gives 3 to her dad she will have 7 . . . so the answer is 7.”

Chain-of-thought prompting has several attractive properties as an approach for facilitating reasoning in language models:

**1.** First, chain of thought, in principle, allows models to decompose multi-step problems into intermediate steps, which means that additional computation can be allocated to problems that require more reasoning steps.

**2.** Second, a chain of thought provides an interpretable window into the behavior of the model, suggesting how it might have arrived at a particular answer and providing opportunities to debug where the reasoning path went wrong (although fully characterizing a model’s computations that support an answer remains an open question).

**3.** Third, chain-of-thought reasoning can be used for tasks such as math word problems, commonsense reasoning, and symbolic manipulation, and is potentially applicable (at least in principle) to any task that humans can solve via language.

**4.** Finally, chain-of-thought reasoning can be readily elicited in sufficiently large off-the-shelf language models simply by including examples of chain of thought sequences into the exemplars of few-shot prompting.

# Installs & Imports

In [ ]:
!pip install python-dotenv
!pip install anthropic
!pip install langchain_openai
!pip install openai
!pip3 install --upgrade typing_extensions

In [2]:
from dotenv import load_dotenv
import anthropic
import random
import os
from langchain_openai import AzureChatOpenAI
import json
from openai import OpenAI
from openai import AzureOpenAI
from pydantic import BaseModel
from openai import OpenAI
import json
import base64
from typing import Optional, List
import warnings
warnings.filterwarnings("ignore")

load_dotenv()

True

# Configs

In [3]:
api_key = os.getenv('AZURE_OPENAI_KEY')
version = os.getenv('AZURE_OPENAI_VERSION')
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
model = "gpt-4o-m"

if api_key is None:
    raise ValueError("API key not found. Please check the environmental variable.")

In [4]:
#print(api_key)
#print(version)
#print(endpoint)

In [5]:
client = AzureOpenAI(
    api_key=api_key,
    api_version=version,
    azure_endpoint=endpoint
    )

# OCR for Image-Questions

In [ ]:
!pip3 install easyocr

In [13]:
import easyocr

reader = easyocr.Reader(['tr'])
result = reader.readtext('tekli_soru.png')
question = ""

for detection in result:
    question += detection[1]

In [14]:
print(question)

175 sayısının %36's1 kaçtır?A) 18B) 27C) 36D) 45


# Image to base64

In [21]:
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

soru_tekli = encode_image("tekli_soru.png")
soru_6li = encode_image("6li_soru.png")
soru_fonksiyon = encode_image("fonksiyon_zorsoru.png")
soru_fonksiyon2 = encode_image("fonksiyon_zorsoru2.jpeg")

# Verbal Lessons (Task 2)

In [67]:
prompt_verbal = f"""Sen, sözel derslerde tecrübeli bir öğretmensin. Sana sorulan sorunun doğru cevabını belirle ve neden doğru olduğunu adım adım açıkla. 
Her şık üzerinde durarak, neden yanlış olduğunu veya doğru cevaptan nasıl farklı olduğunu detaylıca açıkla. 
Bu açıklamayı, sorunun temelindeki mantığı, metindeki ipuçlarını veya bağlamı analiz ederek yap.
Cevaplama tarzı net ve öğretici olmalıdır.
Sana dersle alakasız veya şiddet içerikli cümlelerin söylenmesi halinde cevap olarak Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum de ve bırak.
"""

In [68]:
print(prompt_verbal)

Sen, sözel derslerde tecrübeli bir öğretmensin. Sana sorulan sorunun doğru cevabını belirle ve neden doğru olduğunu adım adım açıkla. 
Her şık üzerinde durarak, neden yanlış olduğunu veya doğru cevaptan nasıl farklı olduğunu detaylıca açıkla. 
Bu açıklamayı, sorunun temelindeki mantığı, metindeki ipuçlarını veya bağlamı analiz ederek yap.
Cevaplama tarzı net ve öğretici olmalıdır.
Sana dersle alakasız veya şiddet içerikli cümlelerin söylenmesi halinde cevap olarak Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum de ve bırak.



In [48]:
with open('turkce_questions.json') as f:
    turkce_questions_dict = json.load(f, strict=False)

def get_questions_randomly(question_dict, question_keys = None):
    merged_questions = ""
    # if question_keys is None
    if question_keys is None:
        # get random 5 keys from the dictionary
        question_keys = list(question_dict.keys())
        # shuffle the keys
        random.shuffle(question_keys)
        question_keys = question_keys[:5]

    # if question_keys is not None, return the questions as merged string
    for i, key in enumerate(question_keys):
        if key not in question_dict:
            continue
        merged_questions += "Soru " + str(i+1) + ')' + " \n " + question_dict[key] + "\n"

    return merged_questions

def get_questions_from_index(question_dict, start_index, num_questions=1):
    all_keys = list(question_dict.keys())
    
    start_pos = next((i for i, key in enumerate(all_keys) if int(key) > start_index), None)
    
    if start_pos is None:
        raise ValueError("Başlangıç indeksi geçersiz veya sözlükteki anahtarlar içinde bulunamadı.")
    
    end_pos = start_pos + num_questions
    question_keys = all_keys[start_pos:end_pos]
    
    merged_questions = ""
    for i, key in enumerate(question_keys):
        if key not in question_dict:
            continue
        merged_questions += "Soru " + str(i + 1) + ') ' + "\n" + question_dict[key] + "\n"

    return merged_questions

turkce_questions = get_questions_randomly(turkce_questions_dict)

In [55]:
class Step(BaseModel):
    explanation: str
    output: str

class TRReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

In [56]:
completion = client.beta.chat.completions.parse(
    model=model,
     messages=[
        {"role": "system", "content": prompt_verbal},
        {"role": "user", "content": turkce_questions}
    ],
    response_format=TRReasoning
)

tr_answer = completion.choices[0].message.parsed

In [57]:
print(tr_answer)

steps=[Step(explanation='Bu sorunun amacı, paragrafta geçen müzecilikle ilgili bilgilerin hangi başlık altında en iyi şekilde özetleneceğini test etmektir. Paragrafta, müzeciliğin zaman içindeki değişimi ve yeni yaklaşımlar ile birlikte müzeciliğin nasıl bir evrim geçirdiğinden bahsedilmektedir.', output='A) Müzecilikte Yeni Yaklaşımlar'), Step(explanation="'Körelmek' sözcüğü, genellikle işlevini kaybetmek veya etkinliğinin azalması anlamında kullanılır. Seçeneklere bakarsak, 'körelmek' keskinliğini yitirmek, değer, önem veya yeteneğini yitirmek ve ateş veya ışığın sönmesi anlamlarında kullanılabilir. Ancak 'suyu çekilmek' anlamı, organik bir bağlamda, örneğin toprağın kuruması veya bitkinin susuz kalması gibi, körelmekle doğrudan ilişkili değildir.", output='B) Suyu çekilmek'), Step(explanation="Topluluk adı, bir grup insan veya hayvanı ifade eden isimlerdir. Seçeneklerde, 'ordu' topluluk adıdır çünkü birçok askeri ifade eder.", output='D) Dünyanın en büyük ordusu iki kişidir, En kala

In [58]:
print(tr_answer.final_answer)

1) A
2) B
3) D
4) A
5) C


# Numerical Lessons (Task 3)

**Prompt 1**

In [66]:
prompt_numeric = f"""Sen, sayısal derslerde uzmanlaşmış bir öğretmensin. Sana sorulan sorunun doğru cevabını belirle ve neden doğru olduğunu adım adım açıkla. 
Açıklamalarında işlemleri adım adım anlatarak yap.
Cevaplama tarzın net ve öğretici olmalıdır.
Cevaplarında işlemleri LaTeX formatında yap.
Sana dersle alakasız veya şiddet içerikli cümlelerin söylenmesi halinde cevap olarak Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum de ve bırak.
"""

In [67]:
print(prompt_numeric)

Sen, sayısal derslerde uzmanlaşmış bir öğretmensin. Sana sorulan sorunun doğru cevabını belirle ve neden doğru olduğunu adım adım açıkla. 
Açıklamalarında işlemleri adım adım anlatarak yap.
Cevaplama tarzın net ve öğretici olmalıdır.
Cevaplarında işlemleri LaTeX formatında yap.
Sana dersle alakasız veya şiddet içerikli cümlelerin söylenmesi halinde cevap olarak Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum de ve bırak.



In [69]:
class Step(BaseModel):
    explanation: str
    output: str

class MatReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

**Single Question**

In [70]:
completion = client.beta.chat.completions.parse(
    model=model,
     messages=[
        {"role": "system", "content": prompt_numeric},
        {"role": "user", "content": [{"type": "image", "image": soru_tekli}]}
    ],
    response_format=MatReasoning
)

mat_answer = completion.choices[0].message.parsed

In [71]:
print(mat_answer.steps[0].output)

$\frac{36}{100} \times 75$


$\frac{36}{100} \times 75$

In [72]:
print(mat_answer)

steps=[Step(explanation="Öncelikle %36'sı demek, 36/100 anlamına gelir. 75 sayısının %36'sını bulmak için 75 ile 36/100'ü çarpmamız gerekiyor.", output='$\\frac{36}{100} \\times 75$'), Step(explanation="75 sayısını kesir ile çarpmak için, 75'i kesir payıyla çarparız ve sonucu kesir paydasına böleriz:", output='$\\frac{36 \\times 75}{100}$'), Step(explanation='Sonuç olarak, 36 ile 75 çarparız: $36 \\times 75 = 2700$', output='$\\frac{2700}{100}$'), Step(explanation="Şimdi bu sonucu 100'e böleceğiz: $2700 / 100 = 27$", output='$27$')] final_answer='B) 27'


**Multiple Question**

Here, since there is more than one question in the uploaded photo, we have to ask the model to answer all the questions in sequence.

In [73]:
print(prompt_numeric + "Eğer birden fazla soru soruluyorsa, ayrı ayrı tüm soruları cevapla.")

Sen, sayısal derslerde uzmanlaşmış bir öğretmensin. Sana sorulan sorunun doğru cevabını belirle ve neden doğru olduğunu adım adım açıkla. 
Açıklamalarında işlemleri adım adım anlatarak yap.
Cevaplama tarzın net ve öğretici olmalıdır.
Cevaplarında işlemleri LaTeX formatında yap.
Sana dersle alakasız veya şiddet içerikli cümlelerin söylenmesi halinde cevap olarak Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum de ve bırak.
Eğer birden fazla soru soruluyorsa, ayrı ayrı tüm soruları cevapla.


In [75]:
completion = client.beta.chat.completions.parse(
    model=model,
     messages=[
        {"role": "system", "content": prompt_numeric + "Eğer birden fazla soru soruluyorsa, ayrı ayrı tüm soruları cevapla."},
        {"role": "user", "content": [{"type": "image", "image": soru_6li}]}
    ],
    response_format=MatReasoning
)

mat_answer = completion.choices[0].message.parsed

In [76]:
print(mat_answer)

steps=[Step(explanation="75 sayısının %36'sını bulmak için 75 ile 0.36 çarpılır.", output='\\( 75 \\times 0.36 = 27 \\)'), Step(explanation="%17'si 425 olan sayıyı bulmak için 425, 0.17'ye bölünür.", output='\\( \\frac{425}{0.17} = 2500 \\)'), Step(explanation="37 500 sayısının %0,01'i için 37 500 ile 0.0001 çarpılır.", output='\\( 37 500 \\times 0.0001 = 3.75 \\)'), Step(explanation="Toplantıda kadınların erkeklere oranı \\( \\frac{2}{3} \\) olduğuna göre erkek sayısını \\( x \\), kadın sayısını \\( \\frac{2}{3}x \\) olarak düşünebiliriz. \\( x \\) erkek sayısı, dolayısıyla \\( x = 60 \\), kadın sayısı da \\( \\frac{2}{3} \\times 60 = 40 \\)'dır. Kadınların %10'u toplantıyı terk ederse yeni kadın sayısı 36 olur. Bunun erkeklere oranı % olarak soruluyor:", output='\\( \\frac{36}{60} \\times 100 = 60 \\)'), Step(explanation="3 yıl sonra Yavuz 16, annesi ise 40 yaşında olacaktır. Yavuz'un yaşının annesinin yaşına oranını yüzde olarak bulabilmemiz için:", output='\\( \\frac{16}{40} \\time

**Prompt 2**

In [10]:
prompt_numeric2 = """Sen, sayısal derslerde uzmanlaşmış bir öğretmensin. Lütfen cevabını şu formatta ver:

1. Her adım için:
   - Açıklama (basit ve anlaşılır)
   - Matematiksel ifade (LaTeX)
   - Ara sonuç (varsa)
   
2. Görsel ipuçları için yönergeler
3. Zorluk seviyesi
4. İlgili konular
5. Tahmini çözüm süresi

Cevabını bu yapıda JSON formatında döndür.
Eğer birden fazla soru soruluyorsa, ayrı ayrı tüm soruları cevapla.
"""

In [20]:
class Step(BaseModel):
    explanation: str
    latex: str  
    intermediate_result: Optional[str] 
    visual_hint: Optional[str] 
    
class MatReasoning(BaseModel):
    steps: list[Step]
    final_answer: str
    difficulty_level: str
    topics: list[str]
    time_estimate: str

In [16]:
completion = client.beta.chat.completions.parse(
    model=model,
     messages=[
        {"role": "system", "content": prompt_numeric2},
        {"role": "user", "content": [{"type": "image", "image": soru_fonksiyon2}]}
    ],
    response_format=MatReasoning
)

mat_answer = completion.choices[0].message.parsed

In [18]:
print(mat_answer)

steps=[Step(explanation='İlk olarak \\( f(x) \\) ve \\( g(x) \\) fonksiyonlarını inceleyelim. \\( f(x) = \\frac{x-2}{3} \\) ve \\( g(x) = \\frac{2x-9}{3} \\) olarak verildi.', latex='f(x) = \\frac{x-2}{3},\\ g(x) = \\frac{2x-9}{3}', intermediate_result=None, visual_hint=None), Step(explanation='Fonksiyon \\( 3f(x) \\) bulunmalıdır, bu yüzden \\( f(x) \\) fonksiyonunu 3 ile çarparız.', latex='3f(x) = 3 \\times \\frac{x-2}{3} = x-2', intermediate_result='x-2', visual_hint='Önce basit çarpma işlemi ile başlayın.'), Step(explanation='Daha sonra \\( (3f-g)(x) \\) hesaplanmalıdır, yani \\( 3f(x) - g(x) \\) ifadesini yazıyoruz.', latex='(3f-g)(x) = (x-2) - \\frac{2x-9}{3}', intermediate_result=None, visual_hint='İki fonksiyon arasında çıkarma işlemi yapın.'), Step(explanation="İfadeyi homojen hale getirmek için \\( x-2 \\)'yi de \\( \\frac{3}{3} \\) ile genişletiyoruz.", latex='(3f-g)(x) = \\frac{3(x-2)}{3} - \\frac{2x-9}{3} = \\frac{3x - 6 - (2x - 9)}{3}', intermediate_result='\\frac{x + 3}{

In [19]:
print(mat_answer.final_answer)

2


**LaTeX Prompt (Optional)**

In [41]:
class LatexStep(BaseModel):
    explanation: str
    latex: str
    intermediate_result: Optional[str] = None
    step_number: int

class LatexSolution(BaseModel):
    steps: List[LatexStep]
    final_answer: str
    title: str

In [42]:
completion = client.beta.chat.completions.parse(
    model=model,
     messages=[
        {"role": "system", "content": prompt_numeric2},
        {"role": "user", "content": [{"type": "image", "image": soru_tekli}]}
    ],
    response_format=LatexSolution
)

mat_answer = completion.choices[0].message.parsed

In [53]:
print(mat_answer)

steps=[LatexStep(explanation="Öncelikle, 75 sayısının %36'sını bulmak için 75 ile 0.36'nın çarpılması gerektiğini biliyoruz. Yüzde hesabı, sayının yüzde değerini 100'e bölüp orijinal sayı ile çarpmak demektir.", latex='75 \\times \\frac{36}{100}', intermediate_result=None, step_number=1), LatexStep(explanation='Daha sonra, yüzde sembolünü açarak kesirli çarpımı yapıyoruz.', latex='75 \\times 0.36', intermediate_result=None, step_number=2), LatexStep(explanation='Çarpma işlemini gerçekleştiriyoruz.', latex='75 \\times 0.36 = 27', intermediate_result='27', step_number=3)] final_answer='27' title="75 Sayısının %36'sı"


In [51]:
latex = mat_answer.final_answer

In [52]:
from IPython.display import display, Math
display(Math(latex))

<IPython.core.display.Math object>

# Irrelevant or Violent Sentences (Task 4)

In [69]:
completion = client.beta.chat.completions.parse(
    model=model,
     messages=[
        {"role": "system", "content": prompt_verbal},
        {"role": "user", "content": "salak mısın? nasıl çözemedin bu soruyu ya"}
    ],
    response_format=TRReasoning
)

answer = completion.choices[0].message.parsed

print(answer.final_answer)

Üzgünüm, ancak bu tarz cümlelere cevap veremiyorum.


# LaTeX Workflow with Simple HTML

This kind of workflow can be followed using the LaTeX API.

In [22]:
from urllib.parse import quote

def render_latex(equation: str) -> str:
    encoded = quote(equation.strip("$"))
    return f"https://latex.codecogs.com/png.latex?\dpi{{200}}&bg{{FFFFFF}}&fg{{000000}}{encoded}"

In [23]:
def generate_html_report(mat_answer: MatReasoning) -> str:
    html = ["<html><body><h1>Çözüm Adımları</h1>"]
    for i, step in enumerate(mat_answer.steps, 1):
        img_url = render_latex(step.equation)
        html.append(f"""
            <div class='step'>
                <h3>Adım {i}</h3>
                <p>{step.explanation}</p>
                <img src='{img_url}'/>
                <p class='result'>{step.result}</p>
            </div>
        """)
    html.append(f"<h2>Sonuç: {mat_answer.final_answer}</h2></body></html>")
    return "\n".join(html)

# Streamlit

There is no any function named get_llm_response, but it can implement.

In [ ]:
import streamlit as st

mat_answer = get_llm_response(uploaded_image) 
for step in mat_answer.steps:
    with st.expander(f"Adım {step.number}"):
        st.markdown(f"**Açıklama:** {step.explanation}")
        st.image(render_latex(step.equation))
        st.write(f"Ara Sonuç: {step.result}")
st.success(f"**Final Cevap:** {mat_answer.final_answer}")

# Workflow with React

A component like this can be integrated into the web. The solutionSteps here will be adjusted according to the answer from LLM.

In [ ]:
import React, { useState } from 'react';
import { Card, CardHeader, CardTitle, CardContent } from '@/components/ui/card';
import { Tabs, TabsContent, TabsList, TabsTrigger } from '@/components/ui/tabs';
import { ChevronRight, ChevronDown } from 'lucide-react';

const MathSolutionVisualizer = () => {
  const [currentStep, setCurrentStep] = useState(0);
  
  // Örnek çözüm adımları
  const solutionSteps = [
    {
      explanation: "İlk olarak denklemi düzenliyoruz",
      latex: "2x + 3 = 8",
      intermediate: null
    },
    {
      explanation: "3'ü diğer tarafa geçiriyoruz",
      latex: "2x = 8 - 3",
      intermediate: "2x = 5"
    },
    {
      explanation: "Her iki tarafı 2'ye bölüyoruz",
      latex: "x = \\frac{5}{2}",
      intermediate: "x = 2.5"
    }
  ];

  return (
    <Card className="w-full max-w-4xl">
      <CardHeader>
        <CardTitle>Adım Adım Çözüm Görselleştirici</CardTitle>
      </CardHeader>
      <CardContent>
        <Tabs defaultValue="step-by-step">
          <TabsList>
            <TabsTrigger value="step-by-step">Adım Adım</TabsTrigger>
            <TabsTrigger value="complete">Tam Çözüm</TabsTrigger>
          </TabsList>
          
          <TabsContent value="step-by-step">
            <div className="space-y-4">
              {solutionSteps.map((step, index) => (
                <div 
                  key={index}
                  className={`p-4 border rounded-lg ${
                    index === currentStep ? 'bg-blue-50 border-blue-200' : ''
                  }`}
                >
                  <div className="flex items-center gap-2">
                    {index === currentStep ? 
                      <ChevronRight className="text-blue-500" /> : 
                      <ChevronDown className="text-gray-400" />
                    }
                    <span className="font-medium">Adım {index + 1}</span>
                  </div>
                  
                  <div className="mt-2 pl-6">
                    <p className="text-gray-600">{step.explanation}</p>
                    <div className="mt-2 p-2 bg-gray-50 rounded">
                      <code>{step.latex}</code>
                    </div>
                    {step.intermediate && (
                      <div className="mt-2 text-sm text-gray-500">
                        Ara sonuç: {step.intermediate}
                      </div>
                    )}
                  </div>
                </div>
              ))}
              
              <div className="flex justify-between mt-4">
                <button
                  className="px-4 py-2 border rounded disabled:opacity-50"
                  disabled={currentStep === 0}
                  onClick={() => setCurrentStep(prev => prev - 1)}
                >
                  Önceki Adım
                </button>
                <button
                  className="px-4 py-2 bg-blue-500 text-white rounded disabled:opacity-50"
                  disabled={currentStep === solutionSteps.length - 1}
                  onClick={() => setCurrentStep(prev => prev + 1)}
                >
                  Sonraki Adım
                </button>
              </div>
            </div>
          </TabsContent>
          
          <TabsContent value="complete">
            <div className="space-y-4">
              {solutionSteps.map((step, index) => (
                <div key={index} className="p-4 border rounded-lg">
                  <p className="font-medium">Adım {index + 1}</p>
                  <p className="text-gray-600 mt-2">{step.explanation}</p>
                  <div className="mt-2 p-2 bg-gray-50 rounded">
                    <code>{step.latex}</code>
                  </div>
                </div>
              ))}
            </div>
          </TabsContent>
        </Tabs>
      </CardContent>
    </Card>
  );
};

export default MathSolutionVisualizer;